In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import yfinance as yf

In [ ]:
def interactive_plot(df, title):
  fig = px.line(title=title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [ ]:
def lstm_stock_predictor(proportion, ticker = "ticker", start_date= "start_date", end_date="end_date"):
  
  # Step 1: grab data
  df = yf.download(tickers=ticker, start=start_date, end=end_date)

  # Step 2: prepare data
  data = pd.DataFrame({'Adj Close': df['Adj Close'], 'Volume': df['Volume']})
  training_data = data.to_numpy()

  # Scaling data, easier for model to learn
  scale_range = MinMaxScaler(feature_range = (0,1))
  training_data_scaled = scale_range.fit_transform(training_data)
  training_data_scaled

  # Set up input/output, e.g predicting next day's price
  X = np.array([training_data_scaled[:-1,0]]).T
  y = training_data_scaled[1:,0]

  # Choosing proportion of data to be trained/tested
  lim = int(proportion * len(X))
  X_train, y_train = X[:lim], y[:lim]
  X_test, y_test = X[lim:], y[lim:]

  X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

  # Step 3: creating model
  inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(optimizer = 'adam', loss = 'mse')

  # Step 4: train model
  history = model.fit(X_train, y_train, epochs =20, batch_size = 32, validation_split = 0.2 )

  # Step 5: final predictions and result
  prediction = model.predict(X)[:,0].tolist()
  scaled_adjclose = training_data_scaled[:-1,0].tolist()

  data_prediction = pd.DataFrame({'Date': df.index[:-1],'Scaled Adj Close': scaled_adjclose, 'Prediction': prediction})

  return interactive_plot(data_prediction, 'og vs pred')